In [46]:
import os

dataset_path = "/home/elsayedmohammed/datasets/5g_nr_positioning_Originla"

new_path = "/home/elsayedmohammed/datasets/5g_nr_positioning"
train_path = os.path.join(new_path, "train")
test_path = os.path.join(new_path, "test")

os.makedirs(train_path, exist_ok=True)
os.makedirs(test_path, exist_ok=True)

In [47]:
import h5py
from sklearn.model_selection import train_test_split
from scipy.io import savemat
random_seed = 42

for base_name in os.listdir(dataset_path):
    if base_name[-2:] == 'py':
        continue
    full_path = os.path.join(dataset_path, base_name)
    print(full_path)

    with h5py.File(full_path, 'r') as data:
        features = data['features'][:]
        positions = data['labels']['position'][:]
        # Split into training and testing sets
    train_features, test_features, train_positions, test_positions = train_test_split(
        features, positions, test_size=0.2, random_state=random_seed
    )

    # Save training set
    train_file = os.path.join(train_path, f"{base_name}")
    with h5py.File(train_file, 'w') as train_data:
        train_data.create_dataset('features', data=train_features)
        train_data.create_dataset('labels/position', data=train_positions)

    # savemat(train_file, {
    #     'features': train_features,
    #     'labels': {'position': train_positions}
    # })
    print(f"Saved training data to: {train_file}")

    # Save testing set
    test_file = os.path.join(test_path, f"{base_name}")
    with h5py.File(test_file, 'w') as test_data:
            test_data.create_dataset('features', data=test_features)
            test_data.create_dataset('labels/position', data=test_positions)
    # savemat(test_file, {
    #     'features': test_features,
    #     'labels': {'position': test_positions}
    # })
    print(f"Saved testing data to: {test_file}")


/home/elsayedmohammed/datasets/5g_nr_positioning_Originla/dataset_SNR20_outdoor.mat
Saved training data to: /home/elsayedmohammed/datasets/5g_nr_positioning/train/dataset_SNR20_outdoor.mat
Saved testing data to: /home/elsayedmohammed/datasets/5g_nr_positioning/test/dataset_SNR20_outdoor.mat
/home/elsayedmohammed/datasets/5g_nr_positioning_Originla/dataset_SNR10_indoor.mat
Saved training data to: /home/elsayedmohammed/datasets/5g_nr_positioning/train/dataset_SNR10_indoor.mat
Saved testing data to: /home/elsayedmohammed/datasets/5g_nr_positioning/test/dataset_SNR10_indoor.mat
/home/elsayedmohammed/datasets/5g_nr_positioning_Originla/dataset_SNR50_indoor.mat
Saved training data to: /home/elsayedmohammed/datasets/5g_nr_positioning/train/dataset_SNR50_indoor.mat
Saved testing data to: /home/elsayedmohammed/datasets/5g_nr_positioning/test/dataset_SNR50_indoor.mat
/home/elsayedmohammed/datasets/5g_nr_positioning_Originla/dataset_SNR50_outdoor.mat
Saved training data to: /home/elsayedmohammed/

In [30]:
from dataset_classes.positioning_nr_dataset import PositioningNR

dataset = PositioningNR(dataset_path, scene="outdoor")

num_samples = len(dataset)
print(num_samples)

11628


In [31]:
from torch.utils.data import random_split
import torch

seed = 42

dataset_train, dataset_test = random_split(dataset, [0.8, 0.2], generator=torch.Generator().manual_seed(seed))

In [32]:
len(dataset_train) + len(dataset_test)

11628

In [33]:
# Function to extract features and labels from a subset
import numpy as np

def extract_data(subset):
    features_list, labels_list = [], []
    for features, labels in subset:
        features_list.append(features.numpy())  # Convert to NumPy array
        labels_list.append(labels.numpy())
    return np.stack(features_list), np.stack(labels_list)

# Extract training and testing data
train_features, train_labels = extract_data(dataset_train)
test_features, test_labels = extract_data(dataset_test)

In [34]:
print(train_features.shape)
print(train_labels.shape)
print(test_features.shape)
print(test_labels.shape)

(9303, 4, 224, 224)
(9303, 3)
(2325, 4, 224, 224)
(2325, 3)


In [ ]:
from scipy.io import savemat

def save_to_multiple_files(features, labels, output_path, prefix, chunk_size=1000):
    """
    Save the data in chunks into multiple .mat files.
    """
    os.makedirs(output_path, exist_ok=True)
    num_samples = features.shape[0]

    for i in range(0, num_samples, chunk_size):
        # Define the chunk range
        start_idx = i
        end_idx = min(i + chunk_size, num_samples)

        # Extract the chunk
        chunk_features = features[start_idx:end_idx]
        chunk_labels = labels[start_idx:end_idx]

        # Save to .mat file
        filename = f"{prefix}_chunk_{start_idx // chunk_size}.mat"
        filepath = os.path.join(output_path, filename)

        savemat(filepath, {
            "features": chunk_features,
            "labels": chunk_labels
        })
        print(f"Saved chunk {start_idx // chunk_size} to {filepath}")

In [ ]:
# Save the splits to .mat files
savemat(os.path.join(train_path, "data.mat"), {
    "features": train_features,
    "labels": train_labels
})
savemat(os.path.join(test_path, "data.mat"), {
    "features": test_features,
    "labels": test_labels
})

OverflowError: Python integer 7468597248 out of bounds for uint32

In [18]:
t.img_size

AttributeError: 'Subset' object has no attribute 'img_size'